In [1]:
pip install pandas Faker

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
from faker import Faker
import random

In [3]:
# Substitua 'nome_do_arquivo.csv' pelo nome do seu arquivo CSV
df_animes = pd.read_csv('Anime_data.csv')

O que temos: Uma tabela de Animes

O que queremos: Uma tabela com Usuarios-Avaliacao de Animes


1- Usar a tabelona de Anime
2 - Usar Criar uma Tabela de Usuario
3 - Criar uma Tabela de Usuario-Anime Avaliacaoes

4- Fazer um join 



In [5]:
def create_fake_users(num:int) -> list[str]:
    '''
        Retorna uma lista de @num de nomes aleatórios de pessoas.
    '''
    return [Faker().name() for _ in range(num)]


def get_nth_most_popular_animes(n:int) -> list[int]:
    animes =df_animes[(df_animes["Type"] == "TV") &( ~ df_animes['Genre'].str.contains("Kids",na=False)) & ( ~ df_animes['Genre'].str.contains("Hentai",na=False)) ]
    most_popular_animes = animes.nlargest(n,"ScoredBy")
    return most_popular_animes.index

def get_rand_animes(max:int) -> list[str]:
    '''
        Retorna uma lista de (no máximo) @max de nomes de animes aleatórios + alguns animes populares

        Meta: Garantir que 1/3 dos animes estejam entre os mais populares.
    '''
    num_animes = random.randint(1,max)
    most_popular_animes = get_nth_most_popular_animes(12)
    rand_anime_indexes = {random.randint(0,len(most_popular_animes)) for i in range(0,num_animes)}
    animes = [df_animes["Title"][i] for i in rand_anime_indexes]

    return animes

In [6]:
def rand_rates(num_users, num_max_animes):
    rates = {}

    for u in create_fake_users(num_users):
        user = {}
        for a in get_rand_animes(num_max_animes):
            rate = round(random.uniform(0, 10), 1)
            user[a] = rate

        rates[u] = user
    
    return rates

rates = rand_rates(25, 7)
df_rates = pd.DataFrame(rates)

print(df_rates, df_rates.info)


                                 Maurice Jones  Sandra Shaw  William Massey  \
Initial D Fourth Stage                     0.0          NaN             NaN   
Naruto                                     8.9          NaN             NaN   
Hungry Heart: Wild Striker                 NaN          8.8             2.8   
Witch Hunter Robin                         NaN          NaN             8.5   
Tennis no Ouji-sama                        NaN          NaN             5.7   
Eyeshield 21                               NaN          NaN             9.1   
Bouken Ou Beet                             NaN          NaN             NaN   
Monster                                    NaN          NaN             NaN   
Cowboy Bebop                               NaN          NaN             NaN   
Cowboy Bebop: Tengoku no Tobira            NaN          NaN             NaN   
Hachimitsu to Clover                       NaN          NaN             NaN   
Trigun                                     NaN      

In [8]:
def cos_between_vectors(A, B):
    # produto interno u e v / |v| * |u|
    norm_a = np.linalg.norm(A)
    norm_b = np.linalg.norm(B)

    if norm_a * norm_b == 0:
        return 0
    
    return A@B / (norm_a * norm_b)

In [9]:
matriz_avaliacoes = df_rates.to_numpy()
# Colunas - Usuarios | Linhas = Anime
matriz_avaliacoes = np.nan_to_num(matriz_avaliacoes)
print(matriz_avaliacoes, "\n\n")
print(matriz_avaliacoes[:,4])

print(cos_between_vectors(matriz_avaliacoes[:, 5], matriz_avaliacoes[:, 9]))

[[ 0.   0.   0.   2.5  3.4  0.   0.   0.   0.   6.8  0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   6.4]
 [ 8.9  0.   0.   0.   0.   0.   0.   9.7  3.1  0.   0.   0.   9.8  0.
   4.2  0.   0.   0.   0.   0.   0.   5.3  0.   0.   3. ]
 [ 0.   8.8  2.8  0.   0.   0.   0.   6.9  0.   4.2  0.   0.   0.   0.
   0.   0.   0.   0.   1.   0.   9.4  0.   3.5  0.   0. ]
 [ 0.   0.   8.5  8.4  0.   9.3  0.   9.3  0.   0.   0.   0.   0.   4.
   0.   9.4  0.   0.   0.   0.   0.   0.   0.   5.6  0.7]
 [ 0.   0.   5.7  0.   3.   0.   0.5  0.   0.   0.1  0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   9.8  0. ]
 [ 0.   0.   9.1 10.   0.   0.   9.   0.   0.   6.9  0.   0.   0.   0.
   0.   9.2  0.   6.5  3.6  0.   0.   9.1  0.   0.   0. ]
 [ 0.   0.   0.   9.9  0.   0.   0.   0.   0.   3.1  0.   0.   0.   6.
   0.   8.8  0.   8.3  0.   8.   0.   1.2  0.   0.   0. ]
 [ 0.   0.   0.   8.3  1.4  0.   5.8  0.   0.   9.8  0.   0.   0.   0.
   0.   0.   0.   0.   6.7

# Sistemas de recomendação Baseado no Collaborative Filtering
### Uma aplicação de álgebra linear

Se você já usou redes sociais, e-commerces ou serviços de streaming, provavelmente um sistema de recomendação fez parte da sua experiência mesmo que discretamente. Esses algoritmos buscam prever qual será a avaliação ou a preferência que um usuário terá sobre outra entidade do sistema, para que possa recomenda-lo algo do seu interesse.

Existem alguns tipos distintos de sistemas de recomendação, e cada um se baseia em uma forma de interação entre usuário-entidade no sistema. Por exemplo: Laura não vive sem escutar suas músicas no cotidiano. O servico de streaming que ela utiliza possui uma funcionalidade que, todo dia pega uma seleção de artistas que se fazem parte dos gêneros que Laura mais se identifica e a apresenta seus trabalhos mais recentes. Com isso, Laura está sempre por dentro das novidades. Outra forma de interação, é a que João tem com o marketplace X. Ele adora meias malucas e sempre utiliza X para comprá-las. Utilizando as avaliações de suas compras anteriores e das avaliações realizadas por outros usuários, o algoritmo da plataforma determina quais são os usuários com gostos parecidos com os de João, para assim, recomenda-lo as meias que agradaram esses usuários semelhantes.

A técnica utilizada pelo marketplace A para recomendar seus produtos aos seus usuários é conhecida como Collaboratve Filtering (CF) e é considerada uma das mais inteligentes formas de recomendação. Como narrado anteriormente, esta técnica se baseia no cálculo da similaridade entre usuários e, a partir dele, se obtém-se uma estimativa do quanto cada um pode gostar de cada item, produto ou até mesmo outros usuários.

O workflow de um sistema que utiliza CF é parecido com:

*   O usuário avalia itens na plataforma, e essas avaliações são interpretadas como uma aproximação do interesse do mesmo naquele tipo de item;
*   Atravéz do interesses dos outros usuários naquele mesmo tipo de item, o sistema mapeia quais usuários são mais similares, isto é, com interesses mais próximos com os daquele usuário;
*   O sistema recomenda itens que foram avaliados pelos usuários similares como "provável interesse" para o usuário que ainda não conhecia tais itens.

A pergunta que fica agora é: Como exatamente se calcula a similariade?
**MATEMÁTICA!** Parece meio esquisito que uma coisa aparentemente subjetiva como o de gosto que dois usuários tem em comum, possa ser expressa numa equação, mas sim, com um olhar de álgebra linear sobre esse problema, podemos obter respostas bastante precisas.

A *Similaridade de cosseno* é uma métrica poderosa para determinar o quão parecido dois objetos são, com a vantagem de não depender do seu tamanho. Nesta técnica, os dados dos objetos são tratados como vetores e dessa forma, conseguimos calcular o cosseno entre os seus ângulos. Pelo fato do cosseno variar entre -1 e 1, podemos usar esta escala para relacionar nossos objetos em um "ambiente controlado". Quanto mais próximo de 1 é o cosseno entre dois objetos, significa que eles são linearmente muito próximos, enquanto quando o cosseno vai se aproximando de -1, pode-se dizer que os objetos também são paralelos, porém tem sentidos opostos. Já quando o cosseno se aproxima de 0, diz-se que os objetos são perpendiculares, e por isso são completamente diferentes e não tem relação alguma. Sua fórmula é dada por:

```
Cos(x, y) = x . y / ||x|| * ||y||
```
onde,

` x . y ` é o produto interno entre os vetores x e y;

`||x||` e `||y||` são as normas de x e y, ou seja, a magnitude dos vetores.



In [4]:

data = {
  'Alan Perlis': {
    'Artificial intelligence': 1.46,
    'Systems programming': 5.0,
    'Software engineering': 3.34,
    'Databases': 2.32
 },
 'Dennis Ritchie': {
    'Programming language theory': 3.45,
    'Systems programming': 5.0,
    'Software engineering': 4.83,
  },
  'Marvin Minsky': {
    'Artificial intelligence': 5.0,
    'Systems programming': 2.54,
    'Computation': 4.32,
    'Algorithms': 2.76
  },
  'John McCarthy': {
    'Artificial intelligence': 5.0,
    'Programming language theory': 4.72,
    'Systems programming': 3.25,
    'Concurrency': 3.61,
    'Formal methods': 3.58,
    'Computation': 3.23,
    'Algorithms': 3.03
  },
  'Edsger Dijkstra': {
    'Programming language theory': 4.34,
    'Systems programming': 4.52,
    'Software engineering': 4.04,
    'Concurrency': 3.97,
    'Formal methods': 5.0,
    'Algorithms': 4.92
  },
  'Donald Knuth': {
    'Programming language theory': 4.33,
    'Systems programming': 3.57,
    'Computation': 4.39,
    'Algorithms': 5.0
  },
  'John Backus': {
    'Programming language theory': 4.58,
    'Systems programming': 4.43,
    'Software engineering': 4.38,
    'Formal methods': 2.42,
    'Databases': 2.80
  },
  'Robert Floyd': {
    'Programming language theory': 4.24,
    'Systems programming': 2.17,
    'Concurrency': 2.92,
    'Formal methods': 5.0,
    'Computation': 3.18,
    'Algorithms': 5.0
  },
  'Tony Hoare': {
    'Programming language theory': 4.64,
    'Systems programming': 4.38,
    'Software engineering': 3.62,
    'Concurrency': 4.88,
    'Formal methods': 4.72,
    'Algorithms': 4.38
  },
  'Edgar Codd': {
    'Systems programming': 4.60,
    'Software engineering': 3.54,
    'Concurrency': 4.28,
    'Formal methods': 1.53,
    'Databases': 5.0
  },
  'Dennis Ritchie': {
    'Programming language theory': 3.45,
    'Systems programming': 5.0,
    'Software engineering': 4.83,
  },
  'Niklaus Wirth': {
    'Programming language theory': 4.23,
    'Systems programming': 4.22,
    'Software engineering': 4.74,
    'Formal methods': 3.83,
    'Algorithms': 3.95
  },
  'Robin Milner': {
    'Programming language theory': 5.0,
    'Systems programming': 1.66,
    'Concurrency': 4.62,
    'Formal methods': 3.94,
  },
  'Leslie Lamport': {
    'Programming language theory': 1.5,
    'Systems programming': 2.76,
    'Software engineering': 3.76,
    'Concurrency': 5.0,
    'Formal methods': 4.93,
    'Algorithms': 4.63
  },
  'Michael Stonebraker': {
    'Systems programming': 4.67,
    'Software engineering': 3.86,
    'Concurrency': 4.14,
    'Databases': 5.0,
  },
    'joao': {
      'Algorithms': 4.63,
      'Concurrency': 3.5,
      'Databases': 3.0,
      'Software engineering': 5.0,
      'Computation': 3.7,
      'Formal methods': 3.8,
      'Programming language theory': 4.1,
      'Systems programming': 3.7
  },
  'thiago': {
    'Algorithms': 4.63,
    'Concurrency': 3.5,
    'Databases': 3.0,
    'Software engineering': 5.0,
  },
   'hashimoto': {
      'Algorithms': 4.5,
      'Computation': 3.0,
      'Formal methods': 3.8,
      'Programming language theory': 5.0,
      'Systems programming': 4.7
  },
  'trindade': {
      'Software engineering': 4.5,
      'Computation': 3.0,
      'Databases': 3.5,
      'Systems programming': 4.7,
      'Formal methods': 1.0,
  },
   'daniel': {
      'Computation': 4.0,
      'Databases': 3.0,
      'Systems programming': 3.5,
      'Formal methods': 1.0,
  },
  'antonio': {
      'Databases': 3.5,
      'Systems programming': 3.5,
      'Formal methods': 4.0,
      'Programming language theory': 5.0,
  },
  'igor': {
      'Databases': 4.0,
      'Concurrency': 3.1,
      'Artificial intelligence': 4.0,
  },
   'zezin': {
      'Systems programming': 4.0,
      'Concurrency': 3.1,
      'Artificial intelligence': 4.0,
      'Algorithms': 3.3
  },
   'landelino': {
      'Computation': 3.0,
      'Algorithms': 4.0
  },
}

data_df = pd.DataFrame(data)
print(data_df)

topicos = [
           'Databases',
           'Systems programming',
           'Formal methods',
           'Programming language theory',
           'Computation',
           'Software engineering',
           'Algorithms',
           'Concurrency',
           'Artificial intelligence',
    ]

In [ ]:
import math

def distancia_euclidiana(pessoa1, pessoa2) :
  itens_comuns = [item for item in data[pessoa1] if item in data[pessoa2]]
  avaliacoes = [(data[pessoa1][item], data[pessoa2][item]) for item in itens_comuns]

  # dist = raiz((x1-x2)² + (y1-y2)²)
  # dist = sum(([avaliacao[0] - avaliacao[1])])

  dist = [pow(avaliacao[0] - avaliacao[1], 2) for avaliacao in avaliacoes]

  return dist

def cosine_similarity(person1, person2):
  itens_comuns = [itm for itm in data[person1] if itm in data[person2]]

  norma_x = sum([pow(data[person1][item], 2) for item in itens_comuns])
  norma_y = sum([pow(data[person2][item], 2) for item in itens_comuns])
  prod_interno_xy = sum([data[person1][item] * data[person2][item] for item in itens_comuns])

  return prod_interno_xy/math.sqrt(norma_x * norma_y)

def pearson_similarity(person1, person2):

	common_ranked_items = [itm for itm in data[person1] if itm in data[person2]]

	n = len(common_ranked_items)

	s1 = sum([data[person1][item] for item in common_ranked_items])
	s2 = sum([data[person2][item] for item in common_ranked_items])

	ss1 = sum([pow(data[person1][item], 2) for item in common_ranked_items])
	ss2 = sum([pow(data[person2][item], 2) for item in common_ranked_items])

	ps = sum([data[person1][item] * data[person2][item] for item in common_ranked_items])

	num = n * ps - (s1 * s2)

	den = math.sqrt((n * ss1 - math.pow(s1, 2)) * (n * ss2 - math.pow(s2, 2)))

	return (num / den) if den != 0 else 0

def recommend1(person, bound, similarity=pearson_similarity):
	scores = [(similarity(person, other), other) for other in data if other != person]

	scores.sort()
	scores.reverse()
	scores = scores[0:bound]

	print (scores)

	recomms = {}

	for sim, other in scores:
		ranked = data[other]

		for itm in ranked:
			if itm not in data[person]:
				weight = sim * ranked[itm]

				if itm in recomms:
					s, weights = recomms[itm]
					recomms[itm] = (s + sim, weights + [weight])
				else:
					recomms[itm] = (sim, [weight])

	for r in recomms:
		sim, item = recomms[r]
		recomms[r] = sum(item) / sim

	return recomms

def recommend2(person, bound, similarity=cosine_similarity):
	scores = [(similarity(person, other), other) for other in data if other != person]

	scores.sort()
	scores.reverse()
	scores = scores[0:bound]

	print (scores)

	recomms = {}

	for sim, other in scores:
		ranked = data[other]

		for itm in ranked:
			if itm not in data[person]:
				weight = sim * ranked[itm]

				if itm in recomms:
					s, weights = recomms[itm]
					recomms[itm] = (s + sim, weights + [weight])
				else:
					recomms[itm] = (sim, [weight])

	for r in recomms:
		sim, item = recomms[r]
		recomms[r] = sum(item) / sim

	return recomms

recommend1('thiago', 20) #pearson_similarity('Dennis Ritchie', 'Alan Perlis')

#recommend2('igor', 20) #cosine_similarity('Dennis Ritchie', 'Alan Perlis')


[(1.0000000000000964, 'zezin'), (1.000000000000005, 'Robert Floyd'), (1.0, 'trindade'), (1.0, 'joao'), (0.9999999999999986, 'John Backus'), (0.999999999999997, 'Alan Perlis'), (0.999999999999962, 'Niklaus Wirth'), (0.3435935963596947, 'Edsger Dijkstra'), (0, 'landelino'), (0, 'hashimoto'), (0, 'daniel'), (0, 'antonio'), (0, 'Robin Milner'), (0, 'Marvin Minsky'), (0, 'Donald Knuth'), (0, 'Dennis Ritchie'), (-0.8436565741460309, 'Michael Stonebraker'), (-0.8711225631152834, 'Leslie Lamport'), (-0.9198254241663395, 'Tony Hoare'), (-0.9629323435498863, 'Edgar Codd')]


{'Artificial intelligence': 2.730000000000063,
 'Computation': 3.293333333333333,
 'Formal methods': 2.957266103504965,
 'Programming language theory': 5.118815690249908,
 'Systems programming': 3.996347539626807}